In [1]:
import numpy as np

# Parámetros del algoritmo
num_hormigas = 10
alfa = 1.0  # Peso de la feromona
beta = 2.0  # Peso de la visibilidad
evaporacion = 0.5  # Tasa de evaporación de la feromona
q0 = 0.8  # Probabilidad de elegir el mejor siguiente paso
iteraciones = 100

# Nueva matriz de distancias
distancias_nueva = np.array([
    [0, 2, 9, 10, 0, 0, 0],
    [2, 0, 1, 6, 0, 0, 0],
    [9, 1, 0, 3, 4, 0, 0],
    [10, 6, 3, 0, 5, 8, 0],
    [0, 0, 4, 5, 0, 7, 2],
    [0, 0, 0, 8, 7, 0, 3],
    [0, 0, 0, 0, 2, 3, 0]
])

# Inicialización de feromonas
feromonas = np.ones_like(distancias_nueva, dtype=np.float64)

# Función para calcular la visibilidad entre ciudades
def calcular_visibilidad(distancias):
    return 1 / np.maximum(distancias, 1e-10)

# Función para seleccionar el próximo paso de la hormiga
def seleccionar_paso(ciudad_actual, feromonas, visibilidad, ciudades_visitadas):
    opciones_validas = [i for i in range(len(feromonas[ciudad_actual, :])) if i not in ciudades_visitadas and distancias_nueva[ciudad_actual, i] > 0]

    if not opciones_validas:
        # No hay opciones válidas, elige una al azar de las no visitadas
        opciones_no_visitadas = [i for i in range(len(feromonas[ciudad_actual, :])) if i not in ciudades_visitadas]
        return np.random.choice(opciones_no_visitadas)
    
    opciones_validas_con_peso = [i for i in opciones_validas if distancias_nueva[ciudad_actual, i] > 0]

    if not opciones_validas_con_peso:
        # Si todas las opciones válidas tienen peso cero, elige una al azar de las no visitadas
        opciones_no_visitadas = [i for i in range(len(feromonas[ciudad_actual, :])) if i not in ciudades_visitadas]
        return np.random.choice(opciones_no_visitadas)

    if np.random.rand() < q0:
        # Selección basada en feromonas y visibilidad
        return opciones_validas_con_peso[np.argmax(feromonas[ciudad_actual, opciones_validas_con_peso] * visibilidad[ciudad_actual, opciones_validas_con_peso])]
    else:
        # Selección probabilística basada en feromonas y visibilidad
        probabilidad = (feromonas[ciudad_actual, opciones_validas_con_peso] ** alfa) * (visibilidad[ciudad_actual, opciones_validas_con_peso] ** beta)
        probabilidad /= np.sum(probabilidad)
        siguiente_ciudad = np.random.choice(opciones_validas_con_peso, p=probabilidad)
        return siguiente_ciudad


# Función para actualizar las feromonas después de cada iteración
def actualizar_feromonas(feromonas, mejores_rutas, evaporacion):
    feromonas *= (1 - evaporacion)
    for ruta in mejores_rutas:
        for i in range(len(ruta) - 1):
            if distancias_nueva[ruta[i], ruta[i + 1]] > 0:
                feromonas[ruta[i], ruta[i + 1]] += 1.0 / len(mejores_rutas)
                feromonas[ruta[i + 1], ruta[i]] += 1.0 / len(mejores_rutas)
    return feromonas

# Función principal del algoritmo
def colonia_hormigas(distancias):
    global feromonas  # Indicar que se utiliza la variable global

    num_ciudades = len(distancias)
    visibilidad = calcular_visibilidad(distancias)

    mejores_rutas_globales = None
    mejor_distancia_global = float('inf')

    for _ in range(iteraciones):
        mejores_rutas_locales = []
        distancias_locales = []

        for _ in range(num_hormigas):
            ciudad_actual = np.random.randint(num_ciudades)
            ciudades_visitadas = [ciudad_actual]
            ruta = [ciudad_actual]
            distancia_total = 0

            for _ in range(num_ciudades - 1):
                siguiente_ciudad = seleccionar_paso(ciudad_actual, feromonas, visibilidad, ciudades_visitadas)
                ciudades_visitadas.append(siguiente_ciudad)
                ruta.append(siguiente_ciudad)
                distancia_total += distancias[ciudad_actual, siguiente_ciudad]
                ciudad_actual = siguiente_ciudad

            mejores_rutas_locales.append(ruta)
            distancias_locales.append(distancia_total)

        feromonas = actualizar_feromonas(feromonas.astype(np.float64), mejores_rutas_locales, evaporacion)

        indice_mejor_local = np.argmin(distancias_locales)
        if distancias_locales[indice_mejor_local] < mejor_distancia_global:
            mejores_rutas_globales = mejores_rutas_locales[indice_mejor_local]
            mejor_distancia_global = distancias_locales[indice_mejor_local]

    return mejores_rutas_globales, mejor_distancia_global

# Uso del algoritmo con la nueva matriz de distancias
mejores_rutas, mejor_distancia = colonia_hormigas(distancias_nueva)
print("Mejor ruta:", mejores_rutas)
print("Mejor distancia:", mejor_distancia)

Mejor ruta: [3, 2, 1, 0, 4, 6, 5]
Mejor distancia: 11
